In [29]:
import pandas as pd
import plotly.express as px
df = pd.read_csv(r'C:/Users/kames/OneDrive/Documents/GitHub/Stout/loans_full_schema.csv')

In [31]:
#Data visualization 1

import plotly.graph_objects as go

locationDf=df.groupby(['state'], as_index=False)[['interest_rate']].mean()

locFig = go.Figure(data=go.Choropleth(
    locations=locationDf['state'],
    z = locationDf['interest_rate'].astype(float),
    locationmode = 'USA-states',
    colorscale = 'Reds',
    colorbar_title = "Average Interest Rate",
))

locFig.update_layout(
    title_text = 'Heatmap of state-wise average interest rate',
    geo_scope='usa',
)

locFig.show()

locFig.write_html('docs/data-visualization-1.1.html')

densityDf=df.groupby(['state'],as_index=False).mean()[['state', 'debt_to_income', 'interest_rate']]

denfig = go.Figure(data=go.Choropleth(
    locations=densityDf['state'],
    z = densityDf['debt_to_income'].astype(float),
    locationmode = 'USA-states',
    colorscale = 'Blues',
    colorbar_title = "Average Debt-To-Income Ratio",
))

denfig.update_layout(
    title_text = 'Heatmap of state-wise debt-to-income ratio',
    geo_scope='usa',
)

denfig.show()

denfig.write_html('docs/data-visualization-1.2.html')

In [32]:
#Data visualization 2

#Preparing Delinquencies and how does it affect interest rate

delinqDf = df[['delinq_2y', 'months_since_last_delinq','account_never_delinq_percent', 'interest_rate']]
df['months_since_last_delinq_categories'] = pd.cut(x=df['months_since_last_delinq'], bins=[0, 20, 40, 60, 80, 100, 120],
                     labels=['0-20', '20-40', '40-60','60-80','80-100','100-120'])
df['account_never_delinq_percent_categories'] = pd.cut(x=df['account_never_delinq_percent'], bins=[0, 20, 40, 60, 80, 100],
                     labels=['0-20', '20-40', '40-60','60-80','80-100'])

figdelinqDf = px.box(df, x="delinq_2y", y="interest_rate")
figdelinqDf.show()
figdelinqDf.write_html('docs/data-visualization-2.1.html')

figLastDel = px.box(df, x="months_since_last_delinq_categories", y="interest_rate")
figLastDel.show()
figLastDel.write_html('docs/data-visualization-2.2.html')

figAcctNotDel = px.box(df, x="account_never_delinq_percent_categories", y="interest_rate")
figAcctNotDel.show()
figAcctNotDel.write_html('docs/data-visualization-2.3.html')

In [40]:
#Data visualization 2

grouping = df.groupby(['issue_month', 'grade'], as_index=False).agg(loan_amount = ('loan_amount', 'mean'), interest_rate = ('interest_rate','mean'))
grouping = grouping.sort_values(by=['issue_month'])
figLoan = px.scatter(grouping, x="loan_amount", y="interest_rate", animation_frame="issue_month", animation_group="grade",
           color="grade",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[0,30])

figLoan.show()

figLoan.write_html('docs/data-visualization-3.1.html')


In [ ]:
#Feature set selection and predicting interest_rate

#Step 1: Data exploration

#We have 10,000 observations on 55 variables.

#1.a We evaluate the missing values columns using the below function

def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table
missing_zero_values_table(df)




In [ ]:
#Based on the above function output we have 11 columns with 0/NAN values. We now evaluate whether these missing values make sense and can they be ignored or imputed

#For num_accounts_120d_past_due column, we have 9682 0 values and 318 missing values. We can impute 0 to the missing values here easily
df.num_accounts_120d_past_due = df.num_accounts_120d_past_due.fillna(0)

#For months_since_last_delinq and months_since_90d_late, there are no 0 values. One can assume that they have no such history and impute 0
df.months_since_last_delinq = df.months_since_last_delinq.fillna(0)
df.months_since_90d_late = df.months_since_90d_late.fillna(0)

#Missing Emp length may be replaced with the median to maintain sanctity of data
df.emp_length = df.emp_length.fillna(df.emp_length.median())

#emp_title, emp_length, debt_to_income and months_since_last_credit_inquiry have low percent of 0/NAN values hence we ignore them for now.

#verification_income_joint, annual_income_joint and debt_to_income_joint have the most missing values each almost 85%.
#These missing  values make sense since they will only be populated when application_type = joint.
countOfJointApplications = df[df.application_type=='joint'].shape[0] #1495
percentOfJointApplication = (countOfJointApplications/df.shape[0])*100 #14.95
#From the above calculation, it is very clear that the number of missing 'joint' type values (mostly 85%) is due to joint applications (14.95%)

#We impute 'Not Applicable' value for verification_income_joint column
df['verification_income_joint'] = df.verification_income_joint.fillna('Not Applicable')

#It is hard to impute the joint income and debt ratio. We cannot consider joint income to be 0 since that would lead to extremely high value from debt ratio
#One way of dealing with this is to split the dataset into two datasets based on application type and model interest rate for each
#But that does not give us a good model. Since this data is MNAR - Missing Not At Random, we will evaluate the outliers for these columns before imputing the value

px.box(x=df.annual_income_joint)


In [ ]:
#1.b Outlier detection and handling

#Our analysis above for missing values led to analyze the outliers first before go back and judge the best way to handle those missing values

num_cols = df.select_dtypes(include=['number'])
Q1 = num_cols.quantile(0.25)
Q3 = num_cols.quantile(0.75)
IQR = Q3 - Q1
((num_cols < (Q1 - 1.5 * IQR)) | (num_cols > (Q3 + 1.5 * IQR))).sum()



In [ ]:
#From above output, we know that we have 68 outliers out of total 1495 in annual_income_joint which is 4.5%. We replace it with the median value given the time constraint

print(df['annual_income_joint'].quantile(0.50)) #113000
print(df['annual_income_joint'].quantile(0.99)) #383417.84
df['annual_income_joint'].loc[(df['annual_income_joint'] > 383417.84)] = 113000.0
px.box(x=df.annual_income_joint)

#Hence, we have dealt with outliers for annual_income_joint. Now we can replace the missing values with mean
df['annual_income_joint'] = df['annual_income_joint'].fillna(df['annual_income_joint'].mean())
px.box(x=df.annual_income_joint)




In [ ]:
#We calculate the debt for each row to impute the debt to income joint ratio
df.debt = df.debt_to_income*df.annual_income
df.debt_to_income_joint = df.debt/df.annual_income_joint
df.debt_to_income_joint.round(2)
px.box(x=df.debt_to_income_joint)

In [ ]:
#Cleaning outlier for debt to income joint
print(df['debt_to_income_joint'].quantile(0.50)) #9.92
print(df['debt_to_income_joint'].quantile(0.99)) #39.20
df['debt_to_income_joint'].loc[(df['debt_to_income_joint'] > 39.20)] = 9.92
px.box(x=df.debt_to_income_joint)

In [ ]:
#After we have accounted for missing values, we replace outliers of all numerical columns to the 5th quantile or 95th quantile
num_col = df.select_dtypes(include=['int64','float64']).columns.tolist()
df[num_col] = df[num_col].apply(lambda x: x.clip(*x.quantile([0.05, 0.95])))

#Since we have few NA values in debt to income columns, we drop them
df.dropna(subset=['debt_to_income', 'debt_to_income_joint'], inplace=True)


In [ ]:
#Step 2: Data sampling into training and testing set

df.drop(columns=['months_since_last_delinq_categories', 'months_since_last_credit_inquiry', 'account_never_delinq_percent_categories'],inplace=True) #Created for plots, months_since_last_credit_inquiry is redundant/less useful variable

modelingDf = df

for col_name in modelingDf.columns:
    if(modelingDf[col_name].dtype == 'object'):
        modelingDf[col_name]= modelingDf[col_name].astype('category')
        modelingDf[col_name] = modelingDf[col_name].cat.codes

df_train = modelingDf.sample(frac=0.7)
df_test = modelingDf.drop(index=df_train.index)

In [ ]:
#Step 3: Creating feature sets
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression

#I use the step-wise selection wrapper method for feature selection


#Before feature selection, drop the highly correlated columns from training set and testing set
correlated_features = set()
correlation_matrix = modelingDf.corr()
for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

df_train.drop(labels=correlated_features, axis=1, inplace=True)
df_test.drop(labels=correlated_features, axis=1, inplace=True)

df_train_X = df_train.drop('interest_rate', 1)
df_train_Y = df_train['interest_rate']

In [ ]:
sffs = SFS(LinearRegression(),
         k_features=(3,15),
         forward=True,
         floating=True,
         cv=4)
features = sffs.fit(df_train_X, df_train_Y)
sffs.k_feature_names_

filtered_features = df_train_X.columns[list(features.k_feature_idx_)]
filtered_features

#We have our 15 most impactful features selected hence we can move on to model selection

In [ ]:
#Step 4: Model Selection

#Algorithm 1: Best Subset Selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def fit_linear_reg(X,Y):
    #Fit linear regression model and return RSS and R squared values
    model_k = LinearRegression(fit_intercept = True)
    model_k.fit(X,Y)
    RSS = mean_squared_error(Y,model_k.predict(X)) * len(Y)
    R_squared = model_k.score(X,Y)
    return RSS, R_squared

In [ ]:
from tqdm import tnrange, tqdm_notebook
import itertools

#Initialization variables
df_train_X_featured = df_train_X[filtered_features]
k = df_train_X_featured.shape[0]
RSS_list, R_squared_list, feature_list = [],[], []
numb_features = []

#Looping over k = 1 to k = 11 features in X
for k in tnrange(1,len(df_train_X_featured.columns) + 1, desc = 'Loop...'):

    #Looping over all possible combinations: from 11 choose k
    for combo in itertools.combinations(df_train_X_featured.columns,k):
        tmp_result = fit_linear_reg(df_train_X_featured[list(combo)],df_train_Y)   #Store temp result 
        RSS_list.append(tmp_result[0])                  #Append lists
        R_squared_list.append(tmp_result[1])
        feature_list.append(combo)
        numb_features.append(len(combo))   

#Store in DataFrame
modelResult1 = pd.DataFrame({'numb_features': numb_features,'RSS': RSS_list, 'R_squared':R_squared_list,'features':feature_list})

In [ ]:
modelResult1['min_RSS'] = modelResult1.groupby('numb_features')['RSS'].transform(min)
modelResult1['max_R_squared'] = modelResult1.groupby('numb_features')['R_squared'].transform(max)
modelResult1.head()

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (16,6))
ax = fig.add_subplot(1, 2, 1)

ax.scatter(modelResult1.numb_features,modelResult1.RSS, alpha = .2, color = 'darkblue' )
ax.set_xlabel('# Features')
ax.set_ylabel('RSS')
ax.set_title('RSS - Best subset selection')
ax.plot(modelResult1.numb_features,modelResult1.min_RSS,color = 'r', label = 'Best subset')
ax.legend()

ax = fig.add_subplot(1, 2, 2)
ax.scatter(modelResult1.numb_features,modelResult1.R_squared, alpha = .2, color = 'darkblue' )
ax.plot(modelResult1.numb_features,modelResult1.max_R_squared,color = 'r', label = 'Best subset')
ax.set_xlabel('# Features')
ax.set_ylabel('R squared')
ax.set_title('R_squared - Best subset selection')
ax.legend()

plt.show()

In [ ]:
#Algorithm 2: OLS method - checking p-values and eliminating non-significant variables
import numpy as np
import statsmodels.api as sm
def get_stats():
    results = sm.OLS(df_train_Y, df_train_X_featured).fit()
    print(results.summary())
get_stats()

#By looking over the p-values of our featured variables, it is clear that the forward selection algorithm used for feature selection gave us a highly efficient model
#All variables have p-value << 0 which shows all these variables are highly significant

In [27]:
#Algorithm 3: Stepwise model selection using AIC value

#FOr this purpose, i will be using R afterextracting data from here

exportDf = modelingDf[filtered_features]
exportDf['interest_rate'] = modelingDf['interest_rate']
exportDf.to_csv('r-input-file.csv')

C:\Users\kames\AppData\Local\Temp\ipykernel_29532\2040416696.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

